In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [2]:
from tqdm import tqdm
import re

In [2]:
from DataAnalysis import DataPreparation as DataPreparation
from DataAnalysis import DataMethodPreparation as DataMethodPreparation
from AssastmentMethod import AspectLaw as AspectLaw
from AssastmentMethod import AspectPhysical as AspectPhysical
from AssastmentMethod import AspectSpatial as AspectSpatial
from AssastmentMethod import AspectEconomic as AspectEconomic

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Загрузка исходного файла с ОКН
heritage_origin = pd.read_csv('Example_files/All_heritage.csv')

C:\Users\Way51\AppData\Local\Temp\ipykernel_18288\1288108848.py:2: DtypeWarning: Columns (1,50) have mixed types. Specify dtype option on import or set low_memory=False.
  heritage_origin = pd.read_csv('Example_files/All_heritage.csv')


In [4]:
# Выборка объектов в нужном регионе
REGION = 'Вологодская область'
ch_region, ch_region_coor, ch_region_none = DataPreparation.separator_reestr(heritage_origin, REGION) # Общий датасет, ГДФ(что можно найти), ДФ без координат оставшиеся

100%|██████████| 975/975 [00:00<00:00, 3539.05it/s]


0    Вологодская область
1    Вологодская область
2    Вологодская область
Name: Регион, dtype: object
Всего объектов: 975
Количество объектов с координатами: 292
Количество объектов без координат: 683


In [87]:
# Загрузка отчета о кадастровой оценке
report = gpd.read_file('Example_files/buildings_report.geojson') 
report = DataPreparation.number_report(report)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\DataAnalysis.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35-164339' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[i, 'учетный номер'] = desc


In [71]:
# УДАЛИТЬ
ch_region['Кадастровый номер'] = ''
for i in range(len(ch_region)):
    for j in range(len(report)):
        if str(ch_region.loc[i, 'Номер в реестре']) == str(report.loc[j, 'Номер в реестре']) or str(ch_region.loc[i, 'учетный номер']) == str(report.loc[j, 'учетный номер']):
            ch_region.loc[i, 'Кадастровый номер'] = str(ch_region.loc[i, 'Кадастровый номер']) + str(report.loc[j, 'Кадастровый номер'])

In [88]:
# Выборка необходимых столбцов для анализа
ch_region = ch_region[['Номер в реестре', 'учетный номер', 'Объект', 'Регион', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'описание предмета охраны', 'Принадлежность к Юнеско', 'наименование документа', 'building_year', 'lat', 'lon', 'geometry']]
report = report[['Номер в реестре','учетный номер', 'Кадастровый номер', 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН', 'Кадастровые номера помещений в объекте недвижимости', 'Площадь, кв.м', 'Назначение здания', 'Год завершения строительства', 'Количество этажей (в том числе подземных)', 'Количество подземных этажей', 'Материал наружных стен', 'Класс конструктивной системы', 'Группа капитальности', 'Объект признан аварийным ', 'Функциональная группа', 'Подгруппа', 'usage', 'area', 'cad_cost', 'geometry']]

# Преобразование столбцов в нужный формат
ch_region['Номер в реестре'] = ch_region['Номер в реестре'].astype(str)
ch_region['учетный номер'] = ch_region['учетный номер'].astype(str)

In [89]:
# Разделение в зависимости от наличие информации
report_1 = report[report['Номер в реестре'] != 0]
report_2 = report[report['Номер в реестре'] == 0]

# объединяем и удалаяем лишние стобцы
gdf_merged_1 = pd.merge(report_1, ch_region, on='Номер в реестре', how='inner')
gdf_merged_2 = pd.merge(report_2, ch_region, on='учетный номер', how='inner')

gdf_merged = pd.concat([gdf_merged_1, gdf_merged_2], ignore_index=True)
gdf_merged = gdf_merged.drop(['Номер в реестре_x', 'Номер в реестре_y', 'учетный номер_y', 'учетный номер_x'], axis = 1)

In [90]:
for i in range(len(gdf_merged)):
    if gdf_merged.loc[i, 'geometry_y'] != None:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_y'].centroid # Если геометрия не пустая, то из полигона делаем точку
    else:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_x']
gdf_merged.rename(columns={'geometry_y':'geometry'}, inplace = True)
gdf_merged = gpd.GeoDataFrame(gdf_merged,geometry='geometry', crs='EPSG:4326')

In [91]:
gdf_merged['geometry']=gdf_merged['geometry'].centroid

C:\Users\Way51\AppData\Local\Temp\ipykernel_18288\1312695513.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_merged['geometry']=gdf_merged['geometry'].centroid


In [92]:
# готовим ДФ для последующего анализа
gdf_merged = DataMethodPreparation.start_all(gdf_merged)

# добавляекм кадастровую стоимость
cad_cost = pd.read_excel('Example_files/report_cost_1.xlsx')
cad_cost = cad_cost[['Кадастровый номер','Аварийность объекта','Экономическое устаревание', 'Эффективный возраст', 'Срок экономической жизни, лет', 'Физический износ', 'Совокупный износ','Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м', 'Кадастровая стоимость (при оценке), руб']]
gdf_merged = pd.merge(gdf_merged, cad_cost, how='left', on='Кадастровый номер')

In [7]:
def parcer_osm(region, tag_str, tag_def = True): #На первом месте указываем тег, на втором его значение по умолчанию выбираются все значения
    tags = {tag_str: tag_def}
    objects = ox.features.features_from_polygon(region.unary_union, tags)
    return objects

def data_osm(city):
    region = ox.geocode_to_gdf(query=city).reset_index()
    building_osm = parcer_osm(region,'building').reset_index()
    shop = parcer_osm(region,'shop').reset_index()
    cafe = parcer_osm(region,'amenity', ['bar', 'cafe', 'fast_food', 'food_court', 'pub', 'restaurant']).reset_index()
    public_transport = parcer_osm(region,'public_transport').reset_index()
    return(region, building_osm, shop, cafe, public_transport)

In [93]:
region, building_osm, shop, cafe, public_transport = data_osm('город Вологда')

gdf_merged = AspectSpatial.start_all(gdf_merged, shop, cafe, public_transport,building_osm)

2025-01-08 22:45:53.758 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:216 - Downloading walk graph from OSM, it may take a while for large territory ...
Calculating the weights of the walk graph: 100%|██████████| 56454/56454 [00:03<00:00, 14985.49it/s]
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\.venv\Lib\site-packages\pyproj\transformer.py:817: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(
2025-01-08 22:46:20.471 | INFO     | objectnat.methods.isochrones:get_accessibility_isochrones:69 - Calculating isochrones distances...


RuntimeError: The point(s) lie further from the graph than weight_value, it's impossible to construct isochrones. Check the coordinates of the point(s)/their projection

In [99]:
# Дополнительная информация из викимапии
wiki_gdf = pd.read_csv('Example_files/file_from_wiki_vologda.csv')
wiki_gdf = gpd.GeoDataFrame(wiki_gdf, geometry=gpd.points_from_xy(wiki_gdf.lon, wiki_gdf.lat), crs='EPSG:4326')

wiki_gdf = wiki_gdf.to_crs('EPSG:32637')
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')

heritage_vologda['geometry'] = heritage_vologda['geometry'].buffer(20)
heritage_vologda['arch_style']=''

for i in range(len(heritage_vologda)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(heritage_vologda.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        heritage_vologda.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

for i in range(len(building_osm)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(building_osm.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        building_osm.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

In [122]:
for i in range(len(building_osm)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(building_osm.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        building_osm.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

In [101]:
gdf_merged = AspectPhysical.start_all(gdf_merged)

In [94]:

gdf_merged = AspectLaw.start_all(gdf_merged)
gdf_merged = AspectEconomic.start_all(gdf_merged)

KeyError: 'score_adastral_integrity'

In [96]:
# Выбираем объекты расположенные в городе Вологда
heritage_vologda = gpd.overlay(gdf_merged, region, how='intersection')

In [113]:
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')
building_osm = building_osm.to_crs('EPSG:32637')

In [11]:
test = heritage_vologda.loc[0:4]

In [12]:
gdf_vision = AspectSpatial.vision(test, building_osm, building_osm)

In [17]:
gdf_vision['geometry'] = gdf_vision['geometry'].buffer(35)

In [114]:
building_osm['geometry'] = building_osm['geometry'].buffer(1)

In [43]:
gdf_vision_1 = gdf_vision[gdf_vision.index == 0]

In [143]:
def merge_reestr_building(gdf, building_osm):
    for i in range(len(building_osm)):
        poly = building_osm.loc[i,'geometry'].buffer(15)
        for j in range(len(gdf)):
            point = gdf.loc[j,'geometry']
            if poly.contains(point):
                building_osm.loc[i,'_building_year'] = gdf.loc[j,'building_year']
                building_osm.loc[i,'_arch_style'] = gdf.loc[j,'arch_style']
                building_osm.loc[i,'_material'] = gdf.loc[j,'material']
                building_osm.loc[i,'_building_floor'] = gdf.loc[j,'Количество этажей (в том числе подземных)']
                building_osm.loc[i,'_culture_heritage'] = 1
    return(building_osm)
                

In [144]:
building_osm = merge_reestr_building(heritage_vologda, building_osm)

In [123]:
def view_selection(gdf_vision, building_osm, gdf):
    gdf['score_historicity'] = 0
    building_osm['geometry'] = building_osm['geometry'].buffer(2)
    for i in range(len(gdf_vision)):
        selection_buildings = gpd.overlay(building_osm, gdf_vision[gdf_vision.index == i], how='intersection')
        if selection_buildings['_culture_heritage'].count() >= 1:
            gdf.loc[i, 'score_historicity'] += 0.1
        floor_mean = selection_buildings['_building_floor'].fillna(1).astype(int).mean()
        if gdf.loc[i, 'Количество этажей (в том числе подземных)'] > (floor_mean - 1) and gdf.loc[i, 'Количество этажей (в том числе подземных)'] < (floor_mean + 1):
            gdf.loc[i, 'score_historicity'] += 0.3
        year_mean = selection_buildings['building_year_1'].fillna(1917).astype(int).mean()
        if gdf.loc[i, 'building_year'] > (year_mean - 20) and gdf.loc[i, 'building_year'] < (year_mean + 20):
            gdf.loc[i, 'score_historicity'] += 0.3
        print(year_mean)
    return(selection_buildings)

In [188]:
type(points_within_polygon_1.groupby(by = ['_arch_style']).count()['geometry'][0])

C:\Users\Way51\AppData\Local\Temp\ipykernel_18288\151280061.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  type(points_within_polygon_1.groupby(by = ['_arch_style']).count()['geometry'][0])


numpy.int64

In [149]:
points_within_polygon_1 = gpd.overlay(building_osm, gdf_vision[gdf_vision.index == 3], how='intersection')

In [ ]:
points_within_polygon_1['_building_floor'].fillna(1).astype(int).mean()

In [172]:
points_within_polygon_1['_building_floor'].fillna(1).astype(int).mean()

1.2321428571428572

In [124]:
a = view_selection(gdf_vision, building_osm)

nan
nan
nan
nan
nan


In [140]:
a.to_file('test.geojson')

In [159]:
points_within_polygon_1['_culture_heritage'].count()

8

In [174]:
points_within_polygon_1.explore('_arch_style')